In [1]:
from os import getcwd

import hdbscan
import matplotlib as mpl
import matplotlib.colors as clr
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma

from astropy.table import Table, Column
from matplotlib.ticker import MultipleLocator
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy.spatial import cKDTree

import megacamlc as mlc

%matplotlib widget

In [2]:
fonts = {'family': 'sans',
         'weight': 'normal',
         'size'  : 16}
mpl.rc('font', **fonts)
mpl.rc('legend', fontsize=14)

In [3]:
apsize = 2.0
basedir = getcwd().split("/")
basedir = "/".join(basedir[:-1])
config = mlc.Config(basedir, "M67", f'apsize_{apsize:.02f}', apsize)

In [4]:
config.analysis

PosixPath('/home/ryan/thesis/cluster/data/M67reduce/apsize_2.00')

In [5]:
allcat = Table.read('../catalogs/M67_allcat.ecsv')
m67 = Table.read('periodograms_clipped.ecsv')

In [6]:
overlap, e78i, alli = np.intersect1d(np.array(m67['gaia_ID']), np.array(allcat['source_id']), return_indices=True)
allcat = allcat[alli]
m67 = m67[e78i]

print(m67.colnames)
print(allcat.colnames)

['gaia_ID', 'lc_period', 'lc_FAP', 'peakratio0', 'peakratio1', 'period1', 'period2', 'see_period', 'see_FAP', 'zpc_period', 'zpc_FAP']
['source_id', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ruwe', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'g', 'r', 'i', 'z', 'g_err', 'r_err', 'i_err', 'z_err', 'prob_kine', 'main_seq', 'binary']


In [7]:
lowfap_allcat = allcat[m67['lc_FAP'] <= 1.]
lowfap_m67 = m67[m67['lc_FAP'] <= 1.]

lowfap_m67 = lowfap_m67[lowfap_allcat['prob_kine'] >= 0.5]
lowfap_allcat = lowfap_allcat[lowfap_allcat['prob_kine'] >= 0.5]

lowfap_allcat = lowfap_allcat[lowfap_m67['peakratio0'] <= 1.1]
lowfap_m67 = lowfap_m67[lowfap_m67['peakratio0'] <= 1.1]

In [8]:
absG = lowfap_allcat['phot_g_mean_mag'] + 5*(1 + np.log10(1.13/1000))
rots = lowfap_m67['lc_period']
probs = lowfap_allcat['prob_kine']
faps = lowfap_m67['lc_FAP']

In [12]:
plotdata = np.c_[np.array(absG), np.array(rots)]
figuretree = cKDTree(plotdata)

In [14]:
fig = plt.figure(figsize=(12, 6))
ax0 = fig.add_subplot(221)
ax0.set_position([0.09, 0.15, 0.385, 0.7])
ax1 = fig.add_subplot(222)
ax1.set_position([0.54, 0.15, 0.385, 0.7])
cax = fig.add_subplot(223)
cax.set_position([0.09, 0.87, 0.385, 0.02])

ax1.tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

ax0.tick_params(which='minor', length=2)
ax0.tick_params(which='major', length=4)
ax0.tick_params(which='both', direction='in')
ax0.yaxis.set_minor_locator(MultipleLocator(2))
ax0.xaxis.set_minor_locator(MultipleLocator(0.5))

ax0.set_ylabel("Period [days]")
ax0.set_xlabel("Absolute G [mag]")
im = ax0.scatter(absG, rots, c=lowfap_m67['peakratio0'], s=8, cmap='copper')
ax0.set_ylim(15, 65)
ax0.set_xlim(6.5, 11.)

def onclick(event):
    clickpoint = np.array([event.xdata, event.ydata])
    d, i = figuretree.query(clickpoint)
    gid = lowfap_allcat['source_id'][i]
    periodogramf = config.analysis / f'periodograms_clipped/mags/{gid}_mags.png'
    im = img.imread(periodogramf)
    ax1.imshow(im)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

cbar = fig.colorbar(im, cax=cax, orientation="horizontal")
cbar.ax.xaxis.set_ticks_position('top')
cbar.ax.xaxis.set_label_position('top')
cbar.set_label("Peak Ratio")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
lowfap_m67[np.abs(lowfap_m67['lc_period']- lowfap_m67['period1']) < 1]

gaia_ID,lc_period,lc_FAP,peakratio0,peakratio1,period1,period2,see_period,see_FAP,zpc_period,zpc_FAP
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
598903090297061760,22.23654941680784,0.009999999999998899,1.024819869382259,1.182035940887628,23.03761946704426,47.74645293136707,11.484785734353254,0.990000000000002,635.1969956153838,0.0
604711505283863808,4.9505761049160615,0.0,1.0851413660135683,1.3251656089007908,4.911706485248626,4.9822378080125445,11.491178601447244,4.169999999999996,645.1219486718742,0.0
604895467323840256,13.590455798222496,0.039999999999995595,1.0887644220375357,1.4032936727886147,13.140612576384452,13.892262690107653,11.484785734353254,1.1099999999999999,655.3619796031737,0.0
604898113023653888,11.656636000846964,0.019999999999997797,1.0773734633509533,1.226858714511493,11.283903994260713,250.22911948484818,11.513609792247616,24.439999999999994,365.3788027876102,0.0
604898555404553344,11.847289731707303,0.16000000000000458,1.0418221017795046,1.2183768148100893,11.462466606052178,22.86146440476188,11.487981278519742,4.530000000000001,635.1969956153838,0.0
604900754427832448,11.433129868314168,0.019999999999997797,1.0218399859952465,1.0539840021566413,11.804989177264293,12.209246454284068,11.484785734353254,0.9600000000000053,645.1219486718742,0.0
604905843964711680,17.25357489134975,0.0,1.0403643839766799,1.5415886050296819,16.77002628554019,18.124585037313413,11.481591967463835,1.3499999999999956,645.1219486718742,0.0
604910581313043584,7.989126299342095,0.0,1.03235100951069,1.618014552077335,8.2509601748601,12.802420066666649,11.491178601447244,6.45,645.1219486718742,0.0
604911886983070720,25.42352507081278,0.0,1.0401889356797545,1.205866449775004,25.918270379786534,24.27266591122866,8.193650469339143,23.329999999999995,371.96220463963914,0.0


In [77]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=150)

ax.hist(lowfap_m67['lc_period']- lowfap_m67['period1'], bins=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  2.,  1.,  1.,
         0.,  0.,  1.,  1.,  1.,  1.,  2.,  1.,  2.,  1.,  4.,  2.,  2.,
         0.,  2.,  2.,  2.,  0.,  1.,  0.,  4.,  3.,  2.,  4.,  2.,  6.,
         5.,  9., 49., 33.,  7.,  4.,  3.,  2.,  0.,  3.,  1.,  1.,  1.,
         2.,  0.,  3.,  2.,  2.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,
         1.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,
         0.,  0.,  0.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  3.]),
 array([-583.22325424, -569.29455246, -555.36585068, -541.4371489 ,
        -527.50844713, -513.57974535, -499.65104357, -485.72234179,
        -471.79364002, -457.86493824, -443.93623646, -430.00753468,
        -416.07883291, -402.15013113, -388.22142935, -374.29272757,
        -360.3640258 , -346.43532402, -332.50662224, -318.57792046,
        -304.64921869, -290.72051691, -276.79181513, -262.86311335,
        -248.93441158, -23

In [32]:
plt.close("all")

In [89]:
rng = np.random.default_rng()

import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(rng.normal(size=100), rng.normal(size=100), 'o')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …